In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import Window
spark = SparkSession.builder.master("local").appName("spark_test").getOrCreate()
    #вычитывает данные из csv
df = spark.read.format("csv")\
    .option("header","true")\
    .option("delimeter",",")\
    .option("inferSchema",True) \
    .load("stocks_price_final.csv")
    #Удаляем из датафрейма лишние колонки
ddf = df.drop('sector','industry','exchange','high','low','close','volume','adjusted','marketcap')
    #делаем нарастающий итог - используется sql решение. Далее поверх него надо уже крутить выбор последней даты
    #Решение должно работать и на кластере, за это отвечает настройка rowsBetween, иначе будет в рамках каждого кластера считать
windowval = (Window.partitionBy('symbol').orderBy('date')
             .rowsBetween(Window.unboundedPreceding, 0))
df_w_cumsum = ddf.withColumn('sum_sum', f.sum('open').over(windowval))
    #Записываем результат в файл. По факту он записывает в отдельный партицированный файл, надо бы еще порыть
df_w_cumsum.repartition(1).write.format("csv").mode("overwrite").option("header","true").save("new_test_3.csv")